## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


__User-User__ рекомендации и __Item-Item__ рекомендации по механизму своего применения идентичны. Вся разница в том что информация об __Items__ как правило менее разреженна и компактна.   

Если есть функция __item_item_rec(interaction_matrix)__ то ее можно использовать и для __user_user_rec__.  


Принципиальные отличия __item-item__ рекомендаций от __ALS__ заключаючаются в следующем:  
__item-item__ рекомендации исходят из предположения что похожие товары распологаются __"близко"__ друг к другу.  
__ALS__ рекомендации предпологают что матрица интеракций избыточна и недостающие значения можно получить при помощи линейных преобразований имеющихся данных.  
Оба предположения часто весьма далеки от реальности.

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


Можно было бы проверить:
1. __Log()__ (или возведение в степень) от суммы или количества покупок. 
2. Попытаться построить взвешивание этих весов с учетом внешней информации о товаре.(скидки, акции, тип товара и т.д.)
3. Попытаться построить взвешивание этих весов с учетом внешней информации о пользователе.(социальный статус, пол и т.д.)

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


Только линейные зависимости. Сам подход к решению задачи через матрицу __(interaction_matrix)__ выбрасывает всю динамику взаимодействия пользователя с товарами и не учитывает порядок, периодичность и стабильность покупок.

4. Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

__BM25__ это функция при помощи которой можно оценить релевантность двух текстов (наборов сущностей). Является модификацией __TFIDF__. Если в __TFIDF__ мы перемножаем __IDF * TF__ то в __BM25__ мы умножаем __IDF__ на модифицированную версию __TF__ (дробь где и в числителе и знаменателе присутствует __TF__ вместе с дополнительными параметрами).   

На основе __BM25__ можно вычислить взаимную близость документов и на основе этой близости отсортировать их, взять топ n-штук и их рекомендовать.

https://ru.wikipedia.org/wiki/Okapi_BM25  
https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables

## Практическая часть


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
import time

def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
        
    return precision

In [2]:
data = pd.read_csv('./data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
result_for_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
result_for_train.columns=['user_id', 'actual']


result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [4]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [5]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\rost_\Anaconda3\envs\recomendation_course\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
user_item_matrix.shape

(2499, 5001)

In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
csr_mat = csr_matrix(user_item_matrix).T.tocsr()

In [9]:
%%time

model = AlternatingLeastSquares(factors=100, #k
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

#recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
#                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
#                        N=5, # кол-во рекомендаций 
#                        filter_already_liked_items=False, 
#                        filter_items=None, 
#                        recalculate_user=True)

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 2.9 s


In [10]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

## Посмотрим какого качества можно добиться на TEST. ##

In [11]:
%%time    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 32.1 s


0.15974534769833498

## Посмотрим какого качества можно добиться на TRAIN. ##

In [12]:
%%time 
result_for_train['als'] = result_for_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result_for_train.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 30.2 s


0.739655862344938

0.15974534769833498 на тесте при 0.739655862344938 на трейне - модель или неадекватна задаче или сильно переучена.

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [13]:
table=[]

mat_1=tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 
mat_2=bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 
mat_3=(mat_1+mat_2)/2
mat_4=(mat_1.toarray()*mat_2.toarray())**(1/2)

mat_list = [mat_1, mat_2, mat_3, mat_4]
titles_list = ['tfidf_weight', 'bm25_weight', 'tfidf_weight and bm25_weight AM', 'tfidf_weight and bm25_weight GM']
for i, mat in enumerate(mat_list):
    table_string=[]
    
    model = AlternatingLeastSquares(factors=100, #k
                                    regularization=0.001,
                                    iterations=15, 
                                    calculate_training_loss=True, 
                                    num_threads=4)

    model.fit(csr_matrix(mat).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    
    table_string.append(titles_list[i])
    result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
    score_test = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
    print("test:", titles_list[i], "score:", score_test)    
    table_string.append(score_test)    
    
    result_for_train['als'] = result_for_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
    score_train = result_for_train.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()    
    print("train:", titles_list[i], "score:", score_train)        
    table_string.append(score_train)    
    
    table.append(table_string) 
    

  0%|          | 0/15 [00:00<?, ?it/s]

test: tfidf_weight score: 0.16307541625857003
train: tfidf_weight score: 0.6497799119647859


  0%|          | 0/15 [00:00<?, ?it/s]

test: bm25_weight score: 0.1922624877571009
train: bm25_weight score: 0.663625450180072


  0%|          | 0/15 [00:00<?, ?it/s]

test: tfidf_weight and bm25_weight AM score: 0.18912830558276203
train: tfidf_weight and bm25_weight AM score: 0.6166466586634654


  0%|          | 0/15 [00:00<?, ?it/s]

test: tfidf_weight and bm25_weight GM score: 0.19147894221351616
train: tfidf_weight and bm25_weight GM score: 0.6488995598239296


In [14]:
table = pd.DataFrame(table, columns=['name', 'test score', 'train score'])    
table

,name,test score,train score
0,tfidf_weight,0.163075,0.649780
1,bm25_weight,0.192262,0.663625
2,tfidf_weight and bm25_weight AM,0.189128,0.616647
3,tfidf_weight and bm25_weight GM,0.191479,0.648900


Вывод - блендинг тут не работает так как значения 0.163075 и 0.192262 слишком сильно разняться друг с другом и так как оба метода взвешивания имеют схожую основу. Тем не менее геометрическое среднее работает лучше арифметического (что и ожидалось).

### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [15]:
table=[]
for factor in [10, 50, int(5000**(1/2)), 100, 300]:
    for l in [0.1, 0.01, 0.001, 0.0001]:
        start = time.perf_counter()
        table_string=[]
        model = AlternatingLeastSquares(factors=factor, #k
                                        regularization=l,
                                        iterations=15, 
                                        calculate_training_loss=True, 
                                        num_threads=4)

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                  show_progress=True)

        elapsed = time.perf_counter()
        elapsed = elapsed - start

        result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        score_test = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
        print("test:","factor:", factor, "regularization:", l, "score:", score_test)    
        table_string.append('test')        
        table_string.append(factor)
        table_string.append(l)    
        table_string.append(score_test)    
        
   
        result_for_train['als'] = result_for_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        score_train = result_for_train.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()    
        print("train:","factor:", factor, "regularization:", l, "score:", score_train)       
        table_string.append(score_train)    
        table_string.append(elapsed)    
        
        table.append(table_string)          
        
        
        
table = pd.DataFrame(table, columns=['name','latent space dim', 'regularization', 'test score', 'train score','elapsed time'])    
table        

  0%|          | 0/15 [00:00<?, ?it/s]

test: factor: 10 regularization: 0.1 score: 0.167384916748286
train: factor: 10 regularization: 0.1 score: 0.5214085634253701


  0%|          | 0/15 [00:00<?, ?it/s]

test: factor: 10 regularization: 0.01 score: 0.166307541625857
train: factor: 10 regularization: 0.01 score: 0.5282112845138055


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

test: factor: 300 regularization: 0.001 score: 0.11772771792360431
train: factor: 300 regularization: 0.001 score: 0.8664265706282515


  0%|          | 0/15 [00:00<?, ?it/s]

test: factor: 300 regularization: 0.0001 score: 0.11762977473065621
train: factor: 300 regularization: 0.0001 score: 0.8677070828331332


,name,latent space dim,regularization,test score,train score,elapsed time
0,test,10,0.1000,0.167385,0.521409,1.798962
1,test,10,0.0100,0.166308,0.528211,1.764429
2,test,10,0.0010,0.163859,0.524370,1.770165
3,test,10,0.0001,0.167287,0.524610,1.818030
4,test,50,0.1000,0.168168,0.626891,1.922630
5,test,50,0.0100,0.166405,0.629452,1.961680
6,test,50,0.0010,0.164055,0.632413,1.935788
7,test,50,0.0001,0.164740,0.632893,1.913007
8,test,70,0.1000,0.165622,0.662985,2.048770
9,test,70,0.0100,0.165622,0.668507,2.096787


**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!

Все выше полученные результаты имеют вероятносную природу поэтому выводы тоже с некоторой степенью вероятности.  
Тем не менее имеем лучший результат __0.168168__ при параметрах __regularization__ __0.1000__ и __latent space dim__ __50__.    
Отсюда вывод - при такой __зверской регуляризации__ и таком __малом скрытом пространсве__ показывать "хорошие" результаты может только заведомо __убогая модель__.